In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("/kaggle/input/logistics-shopee-code-league/delivery_orders_march.csv")

In [ ]:
data

In [ ]:
del_time = np.array([[3,5,7,7],[5,5,7,7],[7,7,7,7],[7,7,7,7]])

In [ ]:
b_add_list = list(data['buyeraddress'])
b_add_list_clean = []

checker = 0
for x in b_add_list:
    if ' metro manila' in x[-14:].lower():
        b_add_list_clean.append('manila')
        checker += 1
    if ' luzon' in x[-14:].lower():
        b_add_list_clean.append('luzon')
        checker += 1
    if ' visayas' in x[-14:].lower():
        b_add_list_clean.append('visayas')
        checker += 1
    if ' mindanao' in x[-14:].lower():
        b_add_list_clean.append('mindanao')
        checker += 1
    if checker == 0:
        print("NO DETECTION FOUND FOR {}".format(x))
    if checker > 1:
        print("DUPLICATES FOUND FOR {}".format(x))
    checker = 0
    

In [ ]:
s_add_list = list(data['selleraddress'])
s_add_list_clean = []

checker = 0
for x in s_add_list:
    if ' metro manila' in x[-14:].lower():
        s_add_list_clean.append('manila')
        checker += 1
    if ' luzon' in x[-14:].lower():
        s_add_list_clean.append('luzon')
        checker += 1
    if ' visayas' in x[-14:].lower():
        s_add_list_clean.append('visayas')
        checker += 1
    if ' mindanao' in x[-14:].lower():
        s_add_list_clean.append('mindanao')
        checker += 1
    if checker == 0:
        print("NO DETECTION FOUND FOR {}".format(x))
    if checker > 1:
        print("DUPLICATES FOUND FOR {}".format(x))
    checker = 0
    

In [ ]:
data['b_address'] = b_add_list_clean

In [ ]:
data['s_address'] = s_add_list_clean

In [ ]:
data

In [ ]:
import datetime
import math

a = min(data['pick'])
print(datetime.datetime.fromtimestamp(a))

a = max(data['pick'])
print(datetime.datetime.fromtimestamp(a))

a = min(data['1st_deliver_attempt'])
print(datetime.datetime.fromtimestamp(a))

a = max(data['1st_deliver_attempt'])
print(datetime.datetime.fromtimestamp(a))

lol = []
for x in data['2nd_deliver_attempt']:
    x = float(x)
    if not math.isnan(x):
        lol.append(x)
        
a = min(lol)
print(datetime.datetime.fromtimestamp(a))
        
a = max(lol)
print(datetime.datetime.fromtimestamp(a))

In [ ]:
import datetime

holidays = ['2020-03-01','2020-03-08','2020-03-15','2020-03-22','2020-03-25','2020-03-29','2020-03-30','2020-03-31','2020-04-05']

holidays = [datetime.datetime.strptime(x, "%Y-%m-%d").date() for x in holidays]

a = datetime.datetime.strptime('2020-04-08', "%Y-%m-%d").date()
a.day - holidays[1].day

In [ ]:
import math
from tqdm import tqdm

first_bool = []
second_bool = [] # True is delivery on time


for i, x in tqdm(enumerate(data['1st_deliver_attempt'])):
    del_dt = datetime.datetime.fromtimestamp(x)
    del_day = del_dt.date()
    
    pickup_dt = datetime.datetime.fromtimestamp(data['pick'][i])
    pickup_day = pickup_dt.date()
    
    
    # For 2nd del date
    dummy_second = data['2nd_deliver_attempt'][i]
    dummy_second = float(dummy_second)
    if math.isnan(dummy_second):
        second_bool.append(True)
    else:
        second_del_day = datetime.datetime.fromtimestamp(data['2nd_deliver_attempt'][i])
        if second_del_day.month == del_day.month:
            second_diff = second_del_day.day - del_day.day
            for dt in holidays:
                if dt.day < second_del_day.day and dt.day >= del_day.day:
                    second_diff -= 1
        elif second_del_day.month > del_day.month:
            second_diff = second_del_day.day - del_day.day + 31
            for dt in holidays:
                if dt.month == del_day.month:
                    if dt.day >= del_day.day:
                        second_diff -= 1
                else:
                    if dt.day < second_del_day.day:
                        second_diff -= 1

        if second_diff > 3:
            second_bool.append(False)
        else:
            second_bool.append(True)
            
    
    
    if del_day.month == pickup_day.month:
        diff = del_day.day - pickup_day.day
        for dt in holidays:
            if dt.day < del_day.day and dt.day >= pickup_day.day:
                diff -= 1
    elif pickup_day.month < del_day.month:
        diff = del_day.day - pickup_day.day + 31
        for dt in holidays:
            if dt.month == pickup_day.month:
                if dt.day >= pickup_day.day:
                    diff -= 1
                else:
                    dt.day < del_day.day:
                        diff -= 1
                    
                
    if data['s_address'][i] == "manila":
        first_idx = 0
    elif data['s_address'][i] == "luzon":
        first_idx = 1
    elif data['s_address'][i] == "visayas":
        first_idx = 2
    elif data['s_address'][i] == "mindanao":
        first_idx = 3
        
    if data['b_address'][i] == "manila":
        second_idx = 0
    elif data['b_address'][i] == "luzon":
        second_idx = 1
    elif data['b_address'][i] == "visayas":
        second_idx = 2
    elif data['b_address'][i] == "mindanao":
        second_idx = 3
        
    limit = del_time[first_idx, second_idx]
    if diff > limit:
        first_bool.append(False)
    else:
        first_bool.append(True)
        
        
                
    
        

In [ ]:
final_bool = []
for i in range(len(first_bool)):
    if first_bool[i] and second_bool[i]:
        final_bool.append(0)
    else:
        final_bool.append(1)
final_bool

In [ ]:
result_data = {'orderid': list(data['orderid']), 'is_late': final_bool}

In [ ]:
result = pd.DataFrame(result_data) 

In [ ]:
result

In [ ]:
result.to_csv("results.csv", index = False)